In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import tensorflow
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
import keras
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.utils import to_categorical


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load Training Data

X_data = np.load('/content/drive/MyDrive/TrainD.npy')
y_labels = np.load('/content/drive/MyDrive/TrainL.npy')
print(X_data.shape)
print(y_labels.shape)

(480, 224, 224, 3)
(480,)


In [5]:
# Build your model
mobilenet = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
model1 = Sequential()
model1.add(mobilenet)
model1.add(Flatten())
model1.add(Dense(2, activation='sigmoid'))

model1.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

9406464/9406464 [==============================] - 0s 0us/step


In [6]:
# Train Model

# Prepare your dataset

# Perform 5-fold cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True)
best_val_acc = 0.0
train_times = []

for fold, (train_indices, val_indices) in enumerate(kfold.split(X_data, y_labels)):
    print(f"Fold: {fold + 1}")

    # Split the data into training and validation sets for the current fold
    X_train_fold, X_val_fold = X_data[train_indices], X_data[val_indices]
    y_train_fold, y_val_fold = y_labels[train_indices], y_labels[val_indices]

    # Perform one-hot encoding of the labels
    num_classes = 2
    y_train_fold = to_categorical(y_train_fold, num_classes)
    y_val_fold = to_categorical(y_val_fold, num_classes)

    # Define a callback to save the best weights based on validation accuracy improvement
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        f"best_weights_mfold{fold + 1}.h5", save_best_only=True, save_weights_only=True, monitor='val_accuracy', mode='max'
    )

    # Define a TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=f"logs/fold{fold + 1}", histogram_freq=1)

    # Train the model
    model1.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=32,
        epochs=20,
        callbacks=[checkpoint_callback, tensorboard_callback]
    )

    # Load the best weights based on validation accuracy
    model1.load_weights(f"best_weights_mfold{fold + 1}.h5")

    # Evaluate the model on the validation set
    _, val_acc = model1.evaluate(X_val_fold, y_val_fold, verbose=0)
    print(f"Validation Accuracy: {val_acc}")

    # Update the best validation accuracy and save the weights if there's an improvement
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        model1.save_weights("best_weights_m.h5")

print("Training complete.")

Fold: 1
Epoch 1/20
12/12 [==============================] - 51s 428ms/step - loss: 0.9493 - accuracy: 0.7839 - val_loss: 1.3472 - val_accuracy: 0.9167
Epoch 2/20
12/12 [==============================] - 3s 294ms/step - loss: 0.2236 - accuracy: 0.9479 - val_loss: 2.1487 - val_accuracy: 0.8750
Epoch 3/20
12/12 [==============================] - 5s 409ms/step - loss: 0.0506 - accuracy: 0.9818 - val_loss: 2.2753 - val_accuracy: 0.8229
Epoch 4/20
12/12 [==============================] - 4s 325ms/step - loss: 0.1006 - accuracy: 0.9792 - val_loss: 2.4765 - val_accuracy: 0.7812
Epoch 5/20
12/12 [==============================] - 3s 291ms/step - loss: 0.0437 - accuracy: 0.9896 - val_loss: 7.3853 - val_accuracy: 0.6354
Epoch 6/20
12/12 [==============================] - 3s 291ms/step - loss: 0.1140 - accuracy: 0.9714 - val_loss: 9.1056 - val_accuracy: 0.7083
Epoch 7/20
12/12 [==============================] - 4s 345ms/step - loss: 0.0923 - accuracy: 0.9818 - val_loss: 6.2569 - val_accuracy: 0.78

In [7]:
# Load Best Weights

model1.load_weights('best_weights_m.h5')

In [8]:
# Load NumPy arrays

X_test = np.load('/content/drive/MyDrive/TestD.npy')
y_test = np.load('/content/drive/MyDrive/TestL.npy')

print(X_test.shape)
print(y_test.shape)

(120, 224, 224, 3)
(120,)


In [9]:
# Confusion Matrix

y_test_pred = model1.predict(X_test)
print('Confusion Matrix (5-fold CV): ')
#test_pred1 = np.argmax(y_test_pred, axis=1)
cm1 = confusion_matrix(np.array(y_test_pred).argmax(axis=1), np.array(y_test))
print(cm1)

#Print accuracy
a1 = round(accuracy_score(np.array(y_test), np.array(y_test_pred).argmax(axis=1)),4)*100
print("Accuracy (5-fold CV): {}".format(a1))

#Classification Report
print("Classification Report (5-fold CV): ")
cr1 = classification_report(np.array(y_test), np.array(y_test_pred).argmax(axis=1))
print(cr1)

4/4 [==============================] - 2s 234ms/step
Confusion Matrix (5-fold CV): 
[[53  2]
 [ 7 58]]
Accuracy (5-fold CV): 92.5
Classification Report (5-fold CV): 
              precision    recall  f1-score   support

           0       0.96      0.88      0.92        60
           1       0.89      0.97      0.93        60

    accuracy                           0.93       120
   macro avg       0.93      0.93      0.92       120
weighted avg       0.93      0.93      0.92       120

